# <span style="color:slateblue"><b>Import Dependencies

In [1]:
# Imports
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import tensorflow as tf
import seaborn as sn
import scipy.stats as sp
import pickle

import plotly.express as px
import plotly.graph_objects as go

# <span style="color:slateblue"><b>Data Load

In [2]:
# file paths
file_path_Raw = "./Resources/cumulative.csv"
file_path_Stellar = "./Resources/stellar_info_final.csv"
file_path_Pickles = "./Pickles/"

In [3]:
# Load raw dataset
with open(file_path_Pickles+'kepler_RAW.pkl', 'rb') as pickle_file:
    keplerRAW_df= pickle.load(pickle_file)

In [4]:
keplerRAW_df

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,9560,10031643,K07984.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,0.765,0.023,-0.541,4.80600,0.63400,-0.63400,87.7,13.0,-13.0,1.11,0.32,-0.23,929.0,NaN,NaN,176.40,152.77,-77.60,8.4,1.0,q1_q17_dr25_tce,5638.0,169.0,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9560,9561,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,1.252,0.051,-0.049,3.22210,0.01740,-0.01740,1579.2,4.6,-4.6,29.35,7.70,-2.57,2088.0,NaN,NaN,4500.53,3406.38,-1175.26,453.3,1.0,q1_q17_dr25_tce,5638.0,139.0,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9561,9562,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,-0.007690,0.043,0.423,-0.043,3.11400,0.22900,-0.22900,48.5,5.4,-5.4,0.72,0.24,-0.08,1608.0,NaN,NaN,1585.81,1537.86,-502.22,10.6,1.0,q1_q17_dr25_tce,6119.0,165.0,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9562,9563,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,0,0.681402,2.434000e-06,-2.434000e-06,132.181750,0.002850,-0.002850,0.147,0.309,-0.147,0.86500,0.16200,-0.16200,103.6,14.7,-14.7,1.07,0.36,-0.11,2218.0,NaN,NaN,5713.41,5675.74,-1836.94,12

In [5]:
# Load stellar dataset
stellar_df = pd.read_csv(file_path_Stellar)
stellar_df

,kepid,koi_smet,koi_smet_err1,koi_smet_err2,koi_smass,koi_smass_err1,koi_smass_err2
0,10797460,0.14,0.15,-0.15,0.919,0.052,-0.046
1,10811496,-0.18,0.30,-0.30,0.961,0.110,-0.121
2,10848459,-0.52,0.30,-0.30,0.836,0.093,-0.077
3,10854555,0.07,0.25,-0.30,1.095,0.151,-0.136
4,10872983,-0.08,0.25,-0.30,1.053,0.135,-0.150
...,...,...,...,...,...,...,...
7805,10090151,0.14,0.20,-0.30,1.005,0.084,-0.115
7806,10128825,-0.04,0.25,-0.30,1.075,0.151,-0.135
7807,10147276,-0.04,0.25,-0.35,1.104,0.146,-0.162
7808,10155286,0.07,0.10,-0.25,2.190,0.225,-0.845


In [6]:
# merge two datasets
rawStellar_df = pd.merge(keplerRAW_df, stellar_df, on=["kepid", "kepid"])
rawStellar_df

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,koi_smet,koi_smet_err1,koi_smet_err2,koi_smass,koi_smass_err1,koi_smass_err2
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,0.14,0.15,-0.15,0.919,0.052,-0.046
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,0.14,0.15,-0.15,0.919,0.052,-0.046
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,-0.18,0.30,-0.30,0.961,0.110,-0.121
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,-0.52,0.30,-0.30,0.836,0.093,-0.077
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,0.07,0.25,-0.30,1.095,0.151,-0.136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9125,9560,10031643,K07984.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,0.765,0.023,-0.541,4.80600,0.63400,-0.63400,87.7,13.0,-13.0,1.11,0.32,-0.23,929.0,NaN,NaN,176.40,152.77,-77.60,8.4,1.0,q1_q17_dr25_tce,5638.0,169.0,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478,-0.16,0.30,-0.25,0.856,0.125,-0.073
9126,9561,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,1.252,0.051,-0.049,3.22210,0.01740,-0.01740,1579.2,4.6,-4.6,29.35,7.70,-2.57,2088.0,NaN,NaN,4500.53,3406.38,-1175.26,453.3,1.0,q1_q17_dr25_tce,5638.0,139.0,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082,0.14,0.20,-0.30,1.005,0.084,-0.115
9127,9562,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,-0.007690,0.043,0.423,-0.043,3.11400,0.22900,-0.22900,48.5,5.4,-5.4,0.72,0.24,-0.08,1608.0,NaN,NaN,1585.81,1537.86,-502.22,10.6,1.0,q1_q17_dr25_tc

# <span style="color:slateblue"><b>EDA & Preprocessing

In [7]:
rawStellar_df.describe()

,rowid,kepid,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,koi_smet,koi_smet_err1,koi_smet_err2,koi_smass,koi_smass_err1,koi_smass_err2
count,9130.000000,9.130000e+03,7900.000000,9130.000000,9130.000000,9130.000000,9130.000000,9130.000000,8986.000000,8986.000000,9130.000000,8986.000000,8986.000000,9075.000000,8986.000000,8986.00000,9130.000000,8986.000000,8986.000000,9.075000e+03,8986.000000,8986.000000,9075.000000,9075.000000,9075.000000,9075.000000,0.0,0.0,9.075000e+03,9.075000e+03,9.075000e+03,9075.000000,8855.000000,9075.000000,9075.000000,9060.000000,9075.000000,9075.000000,9075.000000,9075.000000,9075.000000,9075.000000,9130.000000,9130.000000,9130.000000,9130.000000,9130.000000,9130.000000,9130.000000,9130.000000,9130.000000
mean,4720.050055,7.703657e+06,0.488551,0.171194,0.234392,0.199014,0.123111,74.816471,0.002142,-0.002142,165.742999,0.009804,-0.009804,0.723546,1.921345,-0.32984,5.651114,0.337490,-0.337490,2.286434e+04,115.823548,-115.823548,102.466477,17.742599,-33.387663,1083.873609,NaN,NaN,7.743258e+03,3.728847e+03,-4.021875e+03,260.885697,1.240994,5705.528705,144.639008,-162.213355,4.309502,0.120633,-0.142942,1.716793,0.358691,-0.383164,292.057845,43.827269,14.265396,-0.123817,0.230435,-0.253673,1.023758,0.121923,-0.138445
std,2783.273925,2.649455e+06,0.476974,0.376699,0.423641,0.399281,0.328582,1365.800977,0.008231,0.008231,67.653930,0.022607,0.022607,3.291903,9.336268,1.24779,6.502666,0.662488,0.662488,8.063077e+04,4136.967046,4136.967046,3097.910676,393.612823,1201.760073,857.214319,NaN,NaN,1.605349e+05,5.545282e+04,8.903576e+04,800.449655,0.647467,800.011922,47.020308,72.686519,0.432694,0.132706,0.085054,5.877378,0.891744,1.879375,4.762548,3.598597,1.351490,0.281905,0.074212,0.082458,0.350312,0.085080,0.179155
min,1.000000,7.574500e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.241843,0.000000,-0.172500,120.515914,0.000009,-0.569000,0.000000,0.000000,-59.32000,0.052000,0.000000,-20.200000,0.000000e+00,0.000000,-388600.000000,0.080000,0.000000,-77180.000000,25.000000,NaN,NaN,0.000000e+00,0.000000e+00,-5.600031e+06,0.000000,1.000000,2661.000000,0.000000,-1762.000000,0.047000,0.000000,-1.207000,0.109000,0.000000,-116.137000,279.852720,36.577381,6.966000,-2.500000,0.000000,-0.750000,0.094000,0.000000,-2.432000
25%,2308.250000,5.565489e+06,0.000000,0.000000,0.000000,0.000000,0.000000,2.714568,0.000005,-0.000275,132.754433,0.001250,-0.010500,0.196000,0.040000,-0.44500,2.453000,0.051500,-0.349000,1.590500e+02,9.500000,-48.300000,1.400000,0.240000,-1.980000,537.000000,NaN,NaN,1.964000e+01,1.001000e+01,-2.921650e+02,12.000000,1.000000,5301.000000,106.000000,-198.000000,4.213000,0.042000,-0.196000,0.827000,0.128500,-0.247000,288.660968,40.797556,13.461250,-0.260000,0.150000,-0.300000,0.842000,0.071000,-0.141000
50%,4634.000000,7.938498e+06,0.424000,0.000000,0.000000,0.000000,0.000000,9.431520,0.000035,-0.000035,137.131702,0.004130,-0.004130,0.535000,0.193000,-0.20900,3.817000,0.143000,-0.143000,4.174000e+02,20.500000,-20.500000,2.380000,0.520000,-0.300000,876.000000,NaN,NaN,1.391800e+02,7.580000e+01,-4.184000e+01,22.900000,1.000000,5761.000000,157.000000,-160.000000,4.440000,0.070000,-0.128000,0.997000,0.251000,-0.111000,292.262205,43.706274,14.532000,-0.100000,0.250000,-0.300000,0.972000,0.105000,-0.097000
75%,7174.750000,9.881213e+06,0.999000,0.000000,0.000000,0.000000,0.000000,37.599763,0.000275,-0.000005,170.502038,0.010500,-0.001250,0.886000,0.377875,-0.04600,6.308500,0.349000,-0.051500,1.389650e+03,4

In [8]:
# The default index adds no value, however the "kepoi_name" is the unique identifier for each object of interest (row).
# We make this the index to preserve the relationship through the processing

# change the index to the "kepoi_name" and drop the index title
keplerProcessed_df = rawStellar_df.set_index('kepoi_name')
keplerProcessed_df.rename_axis(None, inplace=True)

keplerProcessed_df 

,rowid,kepid,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,koi_smet,koi_smet_err1,koi_smet_err2,koi_smass,koi_smass_err1,koi_smass_err2
K00752.01,1,10797460,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,0.14,0.15,-0.15,0.919,0.052,-0.046
K00752.02,2,10797460,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,0.14,0.15,-0.15,0.919,0.052,-0.046
K00753.01,3,10811496,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,-0.18,0.30,-0.30,0.961,0.110,-0.121
K00754.01,4,10848459,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,-0.52,0.30,-0.30,0.836,0.093,-0.077
K00755.01,5,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,0.07,0.25,-0.30,1.095,0.151,-0.136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K07984.01,9560,10031643,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,0.765,0.023,-0.541,4.80600,0.63400,-0.63400,87.7,13.0,-13.0,1.11,0.32,-0.23,929.0,NaN,NaN,176.40,152.77,-77.60,8.4,1.0,q1_q17_dr25_tce,5638.0,169.0,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478,-0.16,0.30,-0.25,0.856,0.125,-0.073
K07985.01,9561,10090151,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,1.252,0.051,-0.049,3.22210,0.01740,-0.01740,1579.2,4.6,-4.6,29.35,7.70,-2.57,2088.0,NaN,NaN,4500.53,3406.38,-1175.26,453.3,1.0,q1_q17_dr25_tce,5638.0,139.0,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082,0.14,0.20,-0.30,1.005,0.084,-0.115
K07986.01,9562,10128825,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,-0.007690,0.043,0.423,-0.043,3.11400,0.22900,-0.22900,48.5,5.4,-5.4,0.72,0.24,-0.08,1608.0,NaN,NaN,1585.81,1537.86,-502.22,10.6,1.0,q1_q17_dr25_tce,6119.0,165.0,-220.0,4.444,0.056,-0.224

In [9]:
# Find and drop err columns
columns_to_drop = [col for col in keplerProcessed_df.columns if '_err' in col]

keplerProcessed_df = keplerProcessed_df.drop(columns=columns_to_drop)
keplerProcessed_df

,rowid,kepid,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag,koi_smet,koi_smass
K00752.01,1,10797460,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,35.8,1.0,q1_q17_dr25_tce,5455.0,4.467,0.927,291.93423,48.141651,15.347,0.14,0.919
K00752.02,2,10797460,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,25.8,2.0,q1_q17_dr25_tce,5455.0,4.467,0.927,291.93423,48.141651,15.347,0.14,0.919
K00753.01,3,10811496,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,76.3,1.0,q1_q17_dr25_tce,5853.0,4.544,0.868,297.00482,48.134129,15.436,-0.18,0.961
K00754.01,4,10848459,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,505.6,1.0,q1_q17_dr25_tce,5805.0,4.564,0.791,285.53461,48.285210,15.597,-0.52,0.836
K00755.01,5,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,40.9,1.0,q1_q17_dr25_tce,6031.0,4.438,1.046,288.75488,48.226200,15.509,0.07,1.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K07984.01,9560,10031643,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,1,8.589871,132.016100,0.765,4.80600,87.7,1.11,929.0,176.40,8.4,1.0,q1_q17_dr25_tce,5638.0,4.296,1.088,298.74921,46.973351,14.478,-0.16,0.856
K07985.01,9561,10090151,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,0,0.527699,131.705093,1.252,3.22210,1579.2,29.35,2088.0,4500.53,453.3,1.0,q1_q17_dr25_tce,5638.0,4.529,0.903,297.18875,47.093819,14.082,0.14,1.005
K07986.01,9562,10128825,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,0,1.739849,133.001270,0.043,3.11400,48.5,0.72,1608.0,1585.81,10.6,1.0,q1_q17_dr25_tce,6119.0,4.444,1.031,286.50937,47.163219,14.757,-0.04,1.075
K07987.01,9563,10147276,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,0,0.681402,132.181750,0.147,0.86500,103.6,1.07,2218.0,5713.41,12.3,1.0,q1_q17_dr25_tce,6173.0,4.447,1.041,294.16489,47.176281,15.385,-0.04,1.104


In [10]:
# drop duplicated
keplerProcessed_df = keplerProcessed_df.drop_duplicates(subset=['kepler_name'])
keplerProcessed_df

,rowid,kepid,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag,koi_smet,koi_smass
K00752.01,1,10797460,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,170.538750,0.146,2.9575,615.8,2.26,793.0,93.59,35.8,1.0,q1_q17_dr25_tce,5455.0,4.467,0.927,291.93423,48.141651,15.347,0.14,0.919
K00752.02,2,10797460,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,162.513840,0.586,4.5070,874.8,2.83,443.0,9.11,25.8,2.0,q1_q17_dr25_tce,5455.0,4.467,0.927,291.93423,48.141651,15.347,0.14,0.919
K00753.01,3,10811496,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,175.850252,0.969,1.7822,10829.0,14.60,638.0,39.30,76.3,1.0,q1_q17_dr25_tce,5853.0,4.544,0.868,297.00482,48.134129,15.436,-0.18,0.961
K00755.01,5,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,171.595550,0.701,1.6545,603.3,2.75,1406.0,926.16,40.9,1.0,q1_q17_dr25_tce,6031.0,4.438,1.046,288.75488,48.226200,15.509,0.07,1.095
K00756.01,6,10872983,Kepler-228 d,CONFIRMED,CANDIDATE,1.000,0,0,0,0,11.094321,171.201160,0.538,4.5945,1517.5,3.90,835.0,114.81,66.5,1.0,q1_q17_dr25_tce,6046.0,4.486,0.972,296.28613,48.224670,15.714,-0.08,1.053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K00107.01,9179,11250587,Kepler-464 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,7.256962,134.023923,0.676,4.9402,463.2,3.53,1163.0,432.10,168.3,1.0,q1_q17_dr25_tce,5858.0,4.112,1.549,294.83517,48.982361,12.702,0.22,1.133
K02956.01,9306,8801316,Kepler-1394 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,3.937668,135.277240,0.226,4.6550,24.3,1.36,1947.0,3388.82,16.8,1.0,q1_q17_dr25_tce,6277.0,3.728,2.741,283.45319,45.084141,11.801,-0.06,1.465
K03014.01,9310,7347246,Kepler-1411 b,CONFIRMED,CANDIDATE,0.941,0,0,0,0,86.116089,134.880800,0.016,6.0580,875.2,3.11,441.0,8.93,17.8,1.0,q1_q17_dr25_tce,6161.0,4.454,1.053,286.36157,42.963921,15.831,0.07,1.150
K03106.01,9354,8895758,Kepler-1427 b,CONFIRMED,CANDIDATE,0.877,0,0,0,0,0.968981,132.419210,0.065,1.5170,118.5,1.08,1844.0,2730.51,12.8,1.0,q1_q17_dr25_tce,5866.0,4.473,1.000,295.34967,45.114552,15.415,0.18,1.083


In [11]:
# Rename columns to improve legibility
keplerProcessed_df.rename(columns={'kepid' : 'KepID',
                                   'kepoi_name' : 'koi_name',
                                   'kepler_name' : 'kepler_name',
                                   'koi_disposition' : 'exoplanet_archive_disposition',
                                   'koi_pdisposition' : 'disposition_using_kepler_data',
                                   'koi_score' : 'disposition_score',
                                   'koi_fpflag_nt' : 'not_transit-like_fpf',
                                   'koi_fpflag_ss' : 'stellar_eclipse_fpf',
                                   'koi_fpflag_co' : 'centroid_offset_fpf',
                                   'koi_fpflag_ec' : 'ephemeris_match_indicates',
                                   'koi_period' : 'orbital_period[days]',
                                   'koi_time0bk' : 'transit_epoch',
                                   'koi_impact' : 'impact_parameter',
                                   'koi_duration' : 'transit_duration[hrs]',
                                   'koi_depth' : 'transit_depth[ppm]',
                                   'koi_prad' : 'planetary_radius[earth_radii]',
                                   'koi_teq' : 'equilibrium_temperature[k]',
                                   'koi_insol' : 'insolation_flux[earth_flux]',
                                   'koi_model_snr' : 'transit_signal-to-noise',
                                   'koi_tce_plnt_num' : 'tce_planet_number',
                                   'koi_tce_delivname' : 'tce_delivery',
                                   'koi_steff' : 'stellar_effective_temperature',
                                   'koi_slogg' : 'stellar_surface_gravity',
                                   'koi_smet' : 'stellar_metallicity',
                                   'koi_srad' : 'stellar_radius[solar_radii]',
                                   'koi_smass' : 'stellar_mass[solar_mass]',
                                   'ra' : 'RA[decimal_degrees]',
                                   'dec' : 'Dec[decimal_degrees]',
                                   'koi_kepmag' : 'kepler-band_[mag]'}, inplace = True)
keplerProcessed_df

P:\anaconda3\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,rowid,KepID,kepler_name,exoplanet_archive_disposition,disposition_using_kepler_data,disposition_score,not_transit-like_fpf,stellar_eclipse_fpf,centroid_offset_fpf,ephemeris_match_indicates,orbital_period[days],transit_epoch,impact_parameter,transit_duration[hrs],transit_depth[ppm],planetary_radius[earth_radii],equilibrium_temperature[k],insolation_flux[earth_flux],transit_signal-to-noise,tce_planet_number,tce_delivery,stellar_effective_temperature,stellar_surface_gravity,stellar_radius[solar_radii],RA[decimal_degrees],Dec[decimal_degrees],kepler-band_[mag],stellar_metallicity,stellar_mass[solar_mass]
K00752.01,1,10797460,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,170.538750,0.146,2.9575,615.8,2.26,793.0,93.59,35.8,1.0,q1_q17_dr25_tce,5455.0,4.467,0.927,291.93423,48.141651,15.347,0.14,0.919
K00752.02,2,10797460,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,162.513840,0.586,4.5070,874.8,2.83,443.0,9.11,25.8,2.0,q1_q17_dr25_tce,5455.0,4.467,0.927,291.93423,48.141651,15.347,0.14,0.919
K00753.01,3,10811496,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,175.850252,0.969,1.7822,10829.0,14.60,638.0,39.30,76.3,1.0,q1_q17_dr25_tce,5853.0,4.544,0.868,297.00482,48.134129,15.436,-0.18,0.961
K00755.01,5,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,171.595550,0.701,1.6545,603.3,2.75,1406.0,926.16,40.9,1.0,q1_q17_dr25_tce,6031.0,4.438,1.046,288.75488,48.226200,15.509,0.07,1.095
K00756.01,6,10872983,Kepler-228 d,CONFIRMED,CANDIDATE,1.000,0,0,0,0,11.094321,171.201160,0.538,4.5945,1517.5,3.90,835.0,114.81,66.5,1.0,q1_q17_dr25_tce,6046.0,4.486,0.972,296.28613,48.224670,15.714,-0.08,1.053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K00107.01,9179,11250587,Kepler-464 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,7.256962,134.023923,0.676,4.9402,463.2,3.53,1163.0,432.10,168.3,1.0,q1_q17_dr25_tce,5858.0,4.112,1.549,294.83517,48.982361,12.702,0.22,1.133
K02956.01,9306,8801316,Kepler-1394 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,3.937668,135.277240,0.226,4.6550,24.3,1.36,1947.0,3388.82,16.8,1.0,q1_q17_dr25_tce,6277.0,3.728,2.741,283.45319,45.084141,11.801,-0.06,1.465
K03014.01,9310,7347246,Kepler-1411 b,CONFIRMED,CANDIDATE,0.941,0,0,0,0,86.116089,134.880800,0.016,6.0580,875.2,3.11,441.0,8.93,17.8,1.0,q1_q17_dr25_tce,6161.0,4.454,1.053,286.36157,42.963921,15.831,0.07,1.150
K03106.01,9354,8895758,Kepler-1427 b,CONFIRMED,CANDIDATE,0.877,0,0,0,0,0.968981,132.419210,0.065,1.5170,118.5,1.08,1844.0,2730.51,12.8,1.0,q1_q17_dr25_tce,5866.0,4.473,1.000,295.34967,45.114552,15.415,0.18,1.083


# <span style="color:slateblue"><b> Habitable Zone Analysis

In [12]:
# Store relationships between the id, name, and dispositions it case its needed for future use
Kepler_Exoplanets_df = keplerProcessed_df[["KepID",
                                           "kepler_name",
                                           "exoplanet_archive_disposition",
                                           "disposition_using_kepler_data",
                                           "orbital_period[days]",
                                           "stellar_effective_temperature",
                                           "stellar_surface_gravity",
                                           "stellar_radius[solar_radii]",
                                           "stellar_metallicity",
                                           "stellar_mass[solar_mass]"
                                           ]]
Kepler_Exoplanets_df

,KepID,kepler_name,exoplanet_archive_disposition,disposition_using_kepler_data,orbital_period[days],stellar_effective_temperature,stellar_surface_gravity,stellar_radius[solar_radii],stellar_metallicity,stellar_mass[solar_mass]
K00752.01,10797460,Kepler-227 b,CONFIRMED,CANDIDATE,9.488036,5455.0,4.467,0.927,0.14,0.919
K00752.02,10797460,Kepler-227 c,CONFIRMED,CANDIDATE,54.418383,5455.0,4.467,0.927,0.14,0.919
K00753.01,10811496,NaN,FALSE POSITIVE,FALSE POSITIVE,19.899140,5853.0,4.544,0.868,-0.18,0.961
K00755.01,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,2.525592,6031.0,4.438,1.046,0.07,1.095
K00756.01,10872983,Kepler-228 d,CONFIRMED,CANDIDATE,11.094321,6046.0,4.486,0.972,-0.08,1.053
...,...,...,...,...,...,...,...,...,...,...
K00107.01,11250587,Kepler-464 b,CONFIRMED,CANDIDATE,7.256962,5858.0,4.112,1.549,0.22,1.133
K02956.01,8801316,Kepler-1394 b,CONFIRMED,CANDIDATE,3.937668,6277.0,3.728,2.741,-0.06,1.465
K03014.01,7347246,Kepler-1411 b,CONFIRMED,CANDIDATE,86.116089,6161.0,4.454,1.053,0.07,1.150
K03106.01,8895758,Kepler-1427 b,CONFIRMED,CANDIDATE,0.968981,5866.0,4.473,1.000,0.18,1.083


In [13]:
# Select exoplanet candidate 
Confirmed_Exoplanets_df = Kepler_Exoplanets_df.loc[(Kepler_Exoplanets_df['exoplanet_archive_disposition'] == 'CONFIRMED')
                                                   & (Kepler_Exoplanets_df['disposition_using_kepler_data'] == 'CANDIDATE')]
Confirmed_Exoplanets_df

,KepID,kepler_name,exoplanet_archive_disposition,disposition_using_kepler_data,orbital_period[days],stellar_effective_temperature,stellar_surface_gravity,stellar_radius[solar_radii],stellar_metallicity,stellar_mass[solar_mass]
K00752.01,10797460,Kepler-227 b,CONFIRMED,CANDIDATE,9.488036,5455.0,4.467,0.927,0.14,0.919
K00752.02,10797460,Kepler-227 c,CONFIRMED,CANDIDATE,54.418383,5455.0,4.467,0.927,0.14,0.919
K00755.01,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,2.525592,6031.0,4.438,1.046,0.07,1.095
K00756.01,10872983,Kepler-228 d,CONFIRMED,CANDIDATE,11.094321,6046.0,4.486,0.972,-0.08,1.053
K00756.02,10872983,Kepler-228 c,CONFIRMED,CANDIDATE,4.134435,6046.0,4.486,0.972,-0.08,1.053
...,...,...,...,...,...,...,...,...,...,...
K00107.01,11250587,Kepler-464 b,CONFIRMED,CANDIDATE,7.256962,5858.0,4.112,1.549,0.22,1.133
K02956.01,8801316,Kepler-1394 b,CONFIRMED,CANDIDATE,3.937668,6277.0,3.728,2.741,-0.06,1.465
K03014.01,7347246,Kepler-1411 b,CONFIRMED,CANDIDATE,86.116089,6161.0,4.454,1.053,0.07,1.150
K03106.01,8895758,Kepler-1427 b,CONFIRMED,CANDIDATE,0.968981,5866.0,4.473,1.000,0.18,1.083


In [14]:
# Orbital period
period_exoplanets_df = Confirmed_Exoplanets_df.loc[(Confirmed_Exoplanets_df['orbital_period[days]']>200) 
                                                   & (Confirmed_Exoplanets_df['orbital_period[days]']<400)]
period_exoplanets_df
period_exoplanets_df.count()


KepID                            30
kepler_name                      30
exoplanet_archive_disposition    30
disposition_using_kepler_data    30
orbital_period[days]             30
stellar_effective_temperature    30
stellar_surface_gravity          30
stellar_radius[solar_radii]      30
stellar_metallicity              30
stellar_mass[solar_mass]         30
dtype: int64

In [15]:
# Habitable temperature on these exoplanets

temp_exoplanets_df = Confirmed_Exoplanets_df.loc[(Confirmed_Exoplanets_df['stellar_effective_temperature']>5500) 
                                                 & (Confirmed_Exoplanets_df['stellar_effective_temperature']<6500)]
temp_exoplanets_df

,KepID,kepler_name,exoplanet_archive_disposition,disposition_using_kepler_data,orbital_period[days],stellar_effective_temperature,stellar_surface_gravity,stellar_radius[solar_radii],stellar_metallicity,stellar_mass[solar_mass]
K00755.01,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,2.525592,6031.0,4.438,1.046,0.07,1.095
K00756.01,10872983,Kepler-228 d,CONFIRMED,CANDIDATE,11.094321,6046.0,4.486,0.972,-0.08,1.053
K00756.02,10872983,Kepler-228 c,CONFIRMED,CANDIDATE,4.134435,6046.0,4.486,0.972,-0.08,1.053
K00756.03,10872983,Kepler-228 b,CONFIRMED,CANDIDATE,2.566589,6046.0,4.486,0.972,-0.08,1.053
K00001.01,11446443,Kepler-1 b,CONFIRMED,CANDIDATE,2.470613,5820.0,4.457,0.964,-0.06,0.971
...,...,...,...,...,...,...,...,...,...,...
K00107.01,11250587,Kepler-464 b,CONFIRMED,CANDIDATE,7.256962,5858.0,4.112,1.549,0.22,1.133
K02956.01,8801316,Kepler-1394 b,CONFIRMED,CANDIDATE,3.937668,6277.0,3.728,2.741,-0.06,1.465
K03014.01,7347246,Kepler-1411 b,CONFIRMED,CANDIDATE,86.116089,6161.0,4.454,1.053,0.07,1.150
K03106.01,8895758,Kepler-1427 b,CONFIRMED,CANDIDATE,0.968981,5866.0,4.473,1.000,0.18,1.083


In [16]:
# Metallicity: the proportion of the material of a star or other celestial object that is in elements other than hydrogen or helium.
# Resources in exoplanets

metal_exoplanets_df = Confirmed_Exoplanets_df.loc[(Confirmed_Exoplanets_df['stellar_metallicity']>0)]
metal_exoplanets_df

,KepID,kepler_name,exoplanet_archive_disposition,disposition_using_kepler_data,orbital_period[days],stellar_effective_temperature,stellar_surface_gravity,stellar_radius[solar_radii],stellar_metallicity,stellar_mass[solar_mass]
K00752.01,10797460,Kepler-227 b,CONFIRMED,CANDIDATE,9.488036,5455.0,4.467,0.927,0.14,0.919
K00752.02,10797460,Kepler-227 c,CONFIRMED,CANDIDATE,54.418383,5455.0,4.467,0.927,0.14,0.919
K00755.01,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,2.525592,6031.0,4.438,1.046,0.07,1.095
K00757.01,10910878,Kepler-229 c,CONFIRMED,CANDIDATE,16.068647,5031.0,4.485,0.848,0.16,0.801
K00757.02,10910878,Kepler-229 d,CONFIRMED,CANDIDATE,41.197255,5031.0,4.485,0.848,0.16,0.801
...,...,...,...,...,...,...,...,...,...,...
K02497.01,11179076,Kepler-1261 b,CONFIRMED,CANDIDATE,48.431300,4891.0,4.516,0.805,0.16,0.776
K02589.01,10468885,Kepler-1284 b,CONFIRMED,CANDIDATE,0.664074,5178.0,4.585,0.797,0.16,0.896
K00107.01,11250587,Kepler-464 b,CONFIRMED,CANDIDATE,7.256962,5858.0,4.112,1.549,0.22,1.133
K03014.01,7347246,Kepler-1411 b,CONFIRMED,CANDIDATE,86.116089,6161.0,4.454,1.053,0.07,1.150


In [17]:
# Super-Earth size plante
super_earth_df = Confirmed_Exoplanets_df.loc[(Confirmed_Exoplanets_df['stellar_radius[solar_radii]']<= 2.0)]
super_earth_df

,KepID,kepler_name,exoplanet_archive_disposition,disposition_using_kepler_data,orbital_period[days],stellar_effective_temperature,stellar_surface_gravity,stellar_radius[solar_radii],stellar_metallicity,stellar_mass[solar_mass]
K00752.01,10797460,Kepler-227 b,CONFIRMED,CANDIDATE,9.488036,5455.0,4.467,0.927,0.14,0.919
K00752.02,10797460,Kepler-227 c,CONFIRMED,CANDIDATE,54.418383,5455.0,4.467,0.927,0.14,0.919
K00755.01,10854555,Kepler-664 b,CONFIRMED,CANDIDATE,2.525592,6031.0,4.438,1.046,0.07,1.095
K00756.01,10872983,Kepler-228 d,CONFIRMED,CANDIDATE,11.094321,6046.0,4.486,0.972,-0.08,1.053
K00756.02,10872983,Kepler-228 c,CONFIRMED,CANDIDATE,4.134435,6046.0,4.486,0.972,-0.08,1.053
...,...,...,...,...,...,...,...,...,...,...
K07016.01,8311864,Kepler-452 b,CONFIRMED,CANDIDATE,384.847556,5579.0,4.580,0.798,-0.22,0.892
K00107.01,11250587,Kepler-464 b,CONFIRMED,CANDIDATE,7.256962,5858.0,4.112,1.549,0.22,1.133
K03014.01,7347246,Kepler-1411 b,CONFIRMED,CANDIDATE,86.116089,6161.0,4.454,1.053,0.07,1.150
K03106.01,8895758,Kepler-1427 b,CONFIRMED,CANDIDATE,0.968981,5866.0,4.473,1.000,0.18,1.083


In [18]:
# Confirmed candidates that located in habitable zone
habitable_kepler_df = Confirmed_Exoplanets_df.loc[(Kepler_Exoplanets_df['exoplanet_archive_disposition'] == 'CONFIRMED')
                                                  & (Kepler_Exoplanets_df['disposition_using_kepler_data'] == 'CANDIDATE')
                                                  & (Confirmed_Exoplanets_df['orbital_period[days]']>200) 
                                                  & (Confirmed_Exoplanets_df['orbital_period[days]']<400) 
                                                  & (Confirmed_Exoplanets_df['stellar_effective_temperature']>5500) 
                                                  & (Confirmed_Exoplanets_df['stellar_effective_temperature']<6500)
                                                  & (Confirmed_Exoplanets_df['stellar_radius[solar_radii]']<= 2.0)
                                                  & (Confirmed_Exoplanets_df['stellar_metallicity']>0)
]
habitable_kepler_df.sort_values(by=['kepler_name'], ascending = False)

,KepID,kepler_name,exoplanet_archive_disposition,disposition_using_kepler_data,orbital_period[days],stellar_effective_temperature,stellar_surface_gravity,stellar_radius[solar_radii],stellar_metallicity,stellar_mass[solar_mass]
K00351.02,11442793,Kepler-90 g,CONFIRMED,CANDIDATE,210.601384,5970.0,4.317,1.200,0.10,1.089
K01439.01,11027624,Kepler-849 b,CONFIRMED,CANDIDATE,394.624951,5910.0,4.030,1.727,0.14,1.164
K05581.01,8891684,Kepler-1634 b,CONFIRMED,CANDIDATE,374.878133,5636.0,4.218,1.354,0.42,1.103
K05084.01,4760478,Kepler-1625 b,CONFIRMED,CANDIDATE,287.377314,5548.0,3.964,1.793,0.12,1.079
K04091.01,3103227,Kepler-1550 b,CONFIRMED,CANDIDATE,225.585245,5929.0,4.243,1.363,0.30,1.185
K03946.01,8636434,Kepler-1533 b,CONFIRMED,CANDIDATE,308.544422,6325.0,4.274,1.349,0.14,1.252
K03762.01,11518142,Kepler-1519 b,CONFIRMED,CANDIDATE,240.799047,5758.0,4.168,1.385,0.12,1.029
K03683.01,10795103,Kepler-1515 b,CONFIRMED,CANDIDATE,214.312044,6471.0,4.287,1.357,0.16,1.303
K03681.01,2581316,Kepler-1514 b,CONFIRMED,CANDIDATE,217.831843,6259.0,4.385,1.167,0.14,1.210
K00139.01,8559644,Kepler-111 c,CONFIRMED,CANDIDATE,224.778935,5873.0,4.293,1.235,0.18,1.092


In [19]:
# Pickle results
# Store processed data for use in other notebooks
with open(file_path_Pickles+'kepler_habitable.pkl', 'wb') as pickle_file:
    pickle.dump(habitable_kepler_df, pickle_file)